In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#renamed test_lAUu....csv to test.csv outside program
test= pd.read_csv('test.csv')


In [ ]:
train= pd.read_csv('train.csv')


In [ ]:
train.shape

In [ ]:
test.shape


In [ ]:
train.head()


In [1]:
test

NameError: ignored

In [ ]:
train

In [ ]:
train.isna().sum()


In [ ]:
test.isna().sum()


In [ ]:
test.info()


In [ ]:
train.columns


In [ ]:
for i in ['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status']:
    print(i,'\n')
    print(train[i].value_counts())
    print('')


Replacing missing values

In [ ]:
for i in ['Gender','Married','Dependents', 'Self_Employed', 'Loan_Amount_Term','Credit_History']:
    train[i]=train[i].fillna(train[i].mode()[0])
train['LoanAmount']= train['LoanAmount'].fillna(train['LoanAmount'].median())

In [ ]:
train.isna().sum()


In [ ]:
test.isna().sum()

In [ ]:
for i in ['Gender','Dependents', 'Self_Employed', 'Loan_Amount_Term','Credit_History']:
    test[i]=test[i].fillna(test[i].mode()[0])
test['LoanAmount']= test['LoanAmount'].fillna(test['LoanAmount'].median())

In [ ]:
test.isna().sum()


In [ ]:
train.shape


In [ ]:
test.shape


Modelling

In [ ]:
train= train.drop('Loan_ID',axis=1)
test= test.drop('Loan_ID', axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['Loan_Status']= le.fit_transform(train['Loan_Status'])

In [ ]:
train=pd.get_dummies(train)
test=pd.get_dummies(test)

In [ ]:
train.shape


In [ ]:
test.shape


In [ ]:
X_train=train.drop('Loan_Status', axis=1)
y_train=train['Loan_Status']
X_test=test

In [ ]:
#modelling
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred= rf.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_train

In [ ]:
read= pd.read_csv('test.csv')
read

In [ ]:
#saving dataframe
df= pd.DataFrame({'Loan_ID': read['Loan_ID'], 'Loan_Status':y_pred})

In [ ]:
df['Loan_Status'] = df['Loan_Status'].replace({1: 'Y', 0: 'N'})


In [ ]:
df

In [ ]:
df.info()


In [ ]:
df.isna().sum()


In [ ]:
df['Loan_Status'].value_counts()


In [ ]:
#turning dataframe to csv
df.to_csv('submission(random forest).csv', index=False)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(multi_class='multinomial', max_iter=1000)
model= lr.fit(X_train, y_train)
y_pred = model.predict(X_test)
df= pd.DataFrame({'Loan_ID': read['Loan_ID'], 'Loan_Status':y_pred})
df['Loan_Status'] = df['Loan_Status'].replace({1: 'Y', 0: 'N'})
df.to_csv('submission(Logistic Regression).csv', index=False)

In [ ]:
#svm
from sklearn.svm import SVC
svmclf= SVC(kernel='linear')
model = svmclf.fit(X_train, y_train)
y_pred = svmclf.predict(X_test)
df= pd.DataFrame({'Loan_ID': read['Loan_ID'], 'Loan_Status':y_pred})
df['Loan_Status'] = df['Loan_Status'].replace({1: 'Y', 0: 'N'})
df.to_csv('submission(svm linear).csv', index=False)

In [ ]:
from sklearn.svm import SVC
svmclf= SVC(kernel='poly')
model = svmclf.fit(X_train, y_train)
y_pred = svmclf.predict(X_test)
df= pd.DataFrame({'Loan_ID': read['Loan_ID'], 'Loan_Status':y_pred})
df['Loan_Status'] = df['Loan_Status'].replace({1: 'Y', 0: 'N'})
df.to_csv('submission(svm poly).csv', index=False)

In [ ]:
from sklearn.svm import SVC
svmclf= SVC(kernel='rbf')
model = svmclf.fit(X_train, y_train)
y_pred = svmclf.predict(X_test)
df= pd.DataFrame({'Loan_ID': read['Loan_ID'], 'Loan_Status':y_pred})
df['Loan_Status'] = df['Loan_Status'].replace({1: 'Y', 0: 'N'})
df.to_csv('submission(svm rbf).csv', index=False)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred= dt.predict(X_test)
df= pd.DataFrame({'Loan_ID': read['Loan_ID'], 'Loan_Status':y_pred})
df['Loan_Status'] = df['Loan_Status'].replace({1: 'Y', 0: 'N'})
df.to_csv('submission(decisiontree).csv', index=False)

Hyperparameter Tuning (no need y_test)

In [ ]:
#randomized search
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform, randint

param_dist = {
    'C': uniform(loc=0, scale=1),
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': randint(100, 10000) #randint for low high values
}

# Create a Logistic Regression classifier
lr = LogisticRegression()

# Perform random search
random_search = RandomizedSearchCV(
    lr, param_distributions=param_dist, n_iter=10, cv=5
)

# Fit the random search to find the best hyperparameters
random_search.fit(X_train, y_train)

# Print the best hyperparameters and the associated score
print("Best Hyperparameters: ", random_search.best_params_)
print("Best Score: ", random_search.best_score_)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from scipy.stats import randint

# Define the hyperparameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [10000]
}

# Create a Logistic Regression classifier
lr = LogisticRegression()

# Perform grid search
grid_search = GridSearchCV(
    lr, param_grid=param_grid, cv=5
)

# Fit the grid search to find the best hyperparameters
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the associated score
print("Best Hyperparameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

In [ ]:
#prediction from grid search, since it is best one
best_grid= grid_search.best_estimator_
y_pred= best_grid.predict(X_test)
df= pd.DataFrame({'Loan_ID': read['Loan_ID'], 'Loan_Status':y_pred})
df['Loan_Status'] = df['Loan_Status'].replace({1: 'Y', 0: 'N'})
df.to_csv('lr_gridsearch.csv', index=False)

For RF


In [ ]:
# Define the hyperparameter search space
param_dist = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(1, 10),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'max_features': ['sqrt', 'log2', None]
}

# Create a Random Forest classifier
rf = RandomForestClassifier()

# Perform random search
random_search = RandomizedSearchCV(
    rf, param_distributions=param_dist, n_iter=10, cv=5
)

# Fit the random search to find the best hyperparameters
random_search.fit(X_train, y_train)

# Print the best hyperparameters and the associated score
print("Best Hyperparameters: ", random_search.best_params_)
print("Best Score: ", random_search.best_score_)

In [ ]:
param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Create a Random Forest classifier
rf = RandomForestClassifier()

# Perform grid search
grid_search = GridSearchCV(
    rf, param_grid=param_grid, cv=5
)

# Fit the grid search to find the best hyperparameters
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the associated score
print("Best Hyperparameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

In [ ]:
best_grid= grid_search.best_estimator_
y_pred= best_grid.predict(X_test)
df= pd.DataFrame({'Loan_ID': read['Loan_ID'], 'Loan_Status':y_pred})
df['Loan_Status'] = df['Loan_Status'].replace({1: 'Y', 0: 'N'})
df.to_csv('rf_gridsearch.csv', index=False)

SVM

In [ ]:
# Define the hyperparameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf']
}

# Create an SVM classifier
svm = SVC()

# Perform grid search
grid_search = GridSearchCV(
    svm, param_grid=param_grid, cv=5
)

# Fit the grid search to find the best hyperparameters
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the associated score
print("Best Hyperparameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

In [ ]:
# Define the hyperparameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear']
}

# Create an SVM classifier
svm = SVC()

# Perform grid search
grid_search = GridSearchCV(
    svm, param_grid=param_grid, cv=5
)

# Fit the grid search to find the best hyperparameters
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the associated score
print("Best Hyperparameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)